# Import Important Libraries

In [2]:
import re
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from keras.preprocessing.text import one_hot
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Embedding

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Load Datasets

In [4]:
df = pd.read_csv('news data/train.csv')

In [5]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


# Pre-Processing

In [6]:
### Vocabulary size
voc_size=5000

In [7]:
# Drop rows with missing values
df = df.dropna()

# Split features and label
xTrain =df.drop('label',axis=1)
yTrain = df['label']

print('--- Data Shape ---')
print('xTrain shape: ', xTrain.shape)
print('yTrain shape: ', yTrain.shape)

--- Data Shape ---
xTrain shape:  (18285, 4)
yTrain shape:  (18285,)


### One-Hot Representation

In [8]:
ps = PorterStemmer()

In [9]:
messages = xTrain.copy()

In [10]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [11]:
messages = messages.reset_index()

In [12]:
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [13]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [14]:
onehot_repr = [one_hot(words, voc_size) for words in corpus] 
onehot_repr

[[4141, 730, 2550, 2898, 242, 481, 2564, 3151, 1648, 1040],
 [1198, 4007, 4725, 4937, 655, 620, 2703],
 [4236, 2666, 1639, 4329],
 [1665, 2859, 3772, 3599, 1731, 1996],
 [3585, 655, 4779, 3625, 1842, 665, 655, 3381, 1736, 1906],
 [2970,
  3984,
  2609,
  645,
  3121,
  3092,
  1687,
  2179,
  2464,
  1282,
  669,
  1285,
  171,
  4526,
  2703],
 [398, 4131, 1451, 2956, 3104, 4534, 108, 3243, 458, 2106, 3667],
 [3041, 2024, 4667, 279, 54, 4926, 3092, 4825, 458, 2106, 3667],
 [2014, 3327, 2680, 713, 466, 2269, 284, 2408, 3092, 1282],
 [1428, 880, 1225, 4945, 3783, 1855, 4352, 3182],
 [197, 237, 1498, 1204, 4179, 4574, 391, 96, 877, 2998, 3835],
 [3599, 527, 242, 2269, 3092, 54],
 [4114, 3147, 3088, 4761, 2946, 1450, 2715, 3030, 1914],
 [794, 3380, 3391, 1160, 3416, 4343, 4513, 458, 2106, 3667],
 [2181, 1444, 4346, 2526, 1261, 458, 2106, 3667],
 [2668, 1239, 1591, 4546, 105, 4947, 3618, 3291, 854, 4879],
 [2014, 3142, 4007],
 [4488, 556, 2142, 391, 3092, 2137, 3440, 2703],
 [1713, 4723, 4

### Embedding Representation

In [15]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

In [16]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4141,
        730, 2550, 2898,  242,  481, 2564, 3151, 1648, 1040], dtype=int32)

# Model Definition

In [21]:
embedding_vector_features = 40

model=Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 dropout (Dropout)           (None, 20, 40)            0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [19]:
xTrain = np.array(embedded_docs)
yTrain = np.array(yTrain)

# Model Training

In [20]:
trainX, valX, trainY, valY = train_test_split(xTrain, yTrain, test_size=0.20, random_state=42)

In [23]:
model.fit(trainX, trainY,
          validation_data = (valX, valY),
          epochs=10,
          batch_size=64)

Epoch 1/10
229/229 [==============================] - 28s 88ms/step - loss: 0.3255 - accuracy: 0.8445 - val_loss: 0.1979 - val_accuracy: 0.9125
Epoch 2/10
229/229 [==============================] - 5s 23ms/step - loss: 0.1533 - accuracy: 0.9403 - val_loss: 0.1948 - val_accuracy: 0.9177
Epoch 3/10
229/229 [==============================] - 2s 10ms/step - loss: 0.1155 - accuracy: 0.9584 - val_loss: 0.2081 - val_accuracy: 0.9166
Epoch 4/10
229/229 [==============================] - 2s 10ms/step - loss: 0.0861 - accuracy: 0.9700 - val_loss: 0.2182 - val_accuracy: 0.9119
Epoch 5/10
229/229 [==============================] - 1s 7ms/step - loss: 0.0632 - accuracy: 0.9779 - val_loss: 0.2920 - val_accuracy: 0.9150
Epoch 6/10
229/229 [==============================] - 2s 10ms/step - loss: 0.0505 - accuracy: 0.9837 - val_loss: 0.3157 - val_accuracy: 0.9111
Epoch 7/10
229/229 [==============================] - 2s 10ms/step - loss: 0.0392 - accuracy: 0.9878 - val_loss: 0.3162 - val_accuracy: 0.9128

# Model Validation

In [31]:
predictions = (model.predict(valX) > 0.5).astype("int32")

115/115 [==============================] - 0s 2ms/step


In [33]:
accuracy_score(valY, predictions)

0.9045665846322122

# Save Model

In [35]:
model.save("Fake News Classifier.h5")